In [1]:
import pandas as pd
import numpy as np

In [4]:
house_data=pd.read_csv('kc_house_data.csv')
house_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
house_data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [6]:
#from sklearn.model_selection import train_test_split
#train_data,test_data=train_test_split(house_data,random_state=42,train_size=0.8,test_size=0.2)

In [7]:
train_data=pd.read_csv('kc_house_train_data.csv')
test_data=pd.read_csv('kc_house_test_data.csv')

In [8]:
input_features=train_data['sqft_living']
output=train_data['price']

In [9]:
from sklearn.linear_model import LinearRegression
clf=LinearRegression()

## 1. 
Write a generic function that accepts a column of data (e.g, an SArray) ‘input_feature’ and another column ‘output’ and returns the Simple Linear Regression parameters ‘intercept’ and ‘slope’. 

In [10]:
def simple_linear_regression(input_features,output):
    x=np.array(input_features).reshape(-1,1)
    y=np.array(output).reshape(-1,1)
    clf.fit(x,y)
    return clf.coef_,clf.intercept_

In [11]:
slope,intercept=simple_linear_regression(input_features,output)

In [12]:
slope,intercept

(array([[281.95883963]]), array([-47116.07907289]))

## 2. 
Write a function that accepts a column of data ‘input_feature’, the ‘slope’, and the ‘intercept’ you learned, and returns an a column of predictions ‘predicted_output’ for each entry in the input column. 

In [13]:
def get_regression_predictions(input_feature,intercept=intercept,slope=slope):
    pred_output=[]
    for feature in input_feature:
        out=intercept+(slope*feature)
        pred_output.append(out)
    return pred_output

In [14]:
pred_price=get_regression_predictions(input_features,intercept,slope)
len(pred_price)

17384

## 3.
Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?

In [15]:
pred_2650sqft=get_regression_predictions([2650])
pred_2650sqft

[array([[700074.84594751]])]

## 4.
Write a function that accepts column of data: ‘input_feature’, and ‘output’ and the regression parameters ‘slope’ and ‘intercept’ and outputs the Residual Sum of Squares (RSS).

In [17]:
'''def get_residual_sum_of_squares(input_features,output,intercept=intercept,slope=slope):
    rss=0.0
    for i in range(len(output)):
        pred_price=intercept + (slope*input_features[i])
        rssforone=(pred_price-output[i])**2    # As per this course
        rss=rss + rssforone
        
    return rss'''
def get_residual_sum_of_squares(input_features,output,intercept=intercept,slope=slope):
    input_feature=np.array(input_features).reshape(-1,1)
    out=np.array(output).reshape(-1,1)
    pred_price=intercept+(slope*input_feature)
    error=pred_price-out
    error=error*error
    rss=error.sum(axis=0)        #As per this assignment
        
    return rss


In [18]:
rss=get_residual_sum_of_squares(input_features,output)
rss

array([1.20191835e+15])

## 5.
    Write a function that accept a column of data:‘output’ and the regression parameters ‘slope’ and ‘intercept’ and outputs the column of data: ‘estimated_input’. Do this by solving the linear function output = intercept + slope*input for the ‘input’ variable (i.e. ‘input’ should be on one side of the equals sign by itself). e.g. in python:

In [19]:
def inverse_regression_prediction(output=output,intercept=intercept,slope=slope):
    out=np.array(output).reshape(1,-1)
    input_predict=(out-intercept)/slope
    return input_predict

In [20]:
estimated_input=inverse_regression_prediction()
estimated_input

array([[ 954.09698602, 2075.1826041 ,  805.49373579, ..., 1443.8847869 ,
        1585.74946492, 1319.75319362]])

## 6.
    Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?

In [21]:
estimated_input_800k=inverse_regression_prediction([800000])
estimated_input_800k

array([[3004.39624515]])

## 7.
    Instead of using ‘sqft_living’ to estimate prices we could use ‘bedrooms’ (a count of the number of bedrooms in the house) to estimate prices. Using your function from (3) calculate the Simple Linear Regression slope and intercept for estimating price based on bedrooms. Save this slope and intercept for later (you might want to call them e.g. bedroom_slope, bedroom_intercept).

In [79]:
input_bedrooms=train_data['bedrooms']
bedroom_slope,bedroom_intercept=simple_linear_regression(input_bedrooms,output)
bedroom_slope,bedroom_intercept

(array([[127588.95293399]]), array([109473.17762296]))

In [84]:
input_sqft_test=test_data['sqft_living']
input_bedrooms_test=test_data['bedrooms']
output_test=test_data['price']

## 8.
    Now that we have 2 different models compute the RSS from BOTH models on TEST data.

In [85]:
rss_sqft=get_residual_sum_of_squares(input_sqft_test,output_test)
rss_sqft

array([2.75402934e+14])

In [86]:
rss_bedrooms=get_residual_sum_of_squares(input_bedrooms_test,output_test,bedroom_intercept,bedroom_slope)
rss_bedrooms

array([4.93364586e+14])